<a href="https://colab.research.google.com/github/ewertonsilvag/mini_projeto_final_SoulCode/blob/main/Insights_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INSTALANDO O PYSPARK


In [ ]:
!pip install pyspark

IMPORTANDO BIBLIOTECAS

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

CRIANDO UMA CONEXAO COM O SPARK

In [ ]:
spark = (SparkSession.builder\
        .master("local")\
        .appName("mini-projeto")\
        .config("spark.ui.port", "4050")\
        .getOrCreate())

Importando e configurando o database

In [ ]:
df = (spark
      .read
      .format("csv")
      .option("header", "true")
      .option("inferschema", "true")
      .option("delimiter", ",")
      .load("/content/drive/MyDrive/SoulCode/MiniProjeto/df_marketing_pySpark")
      .createOrReplaceTempView("marketing"))

Descrevendo o data frame

In [ ]:
spark.sql('''
DESCRIBE marketing
''').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|       identificacao|      int|   null|
|      ano_nascimento|      int|   null|
|            educacao|   string|   null|
|        estado_civil|   string|   null|
|               renda|   double|   null|
|         qtd_crianca|      int|   null|
|     qtd_adolescente|      int|   null|
|     primeira_compra|   string|   null|
|dias_desde_ult_co...|      int|   null|
|          qtd_vinhos|      int|   null|
|          qtd_frutas|      int|   null|
|           qtd_carne|      int|   null|
|           qtd_peixe|      int|   null|
|          guloseimas|      int|   null|
|   qtd_produtos_gold|      int|   null|
|  qtd_ofertas_compra|      int|   null|
|  qtd_compras_online|      int|   null|
|qtd_compras_catalogo|      int|   null|
| qtd_compras_na_loja|      int|   null|
|qtd_visitas_onlin...|      int|   null|
+--------------------+---------+-------+
only showing top

Mostrando todas as colunas do data frame

In [ ]:
spark.sql('''SELECT * FROM marketing''').show()

+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+--------------+----------+
|identificacao|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|comprou_online|qtd_filhos|
+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------

Contando de forma distinta a quantidade de clientes em relação ao estado civil

In [ ]:
#Seria interessante o mercado saber que tipos de consumidores eles estão atendendo para
#poderem direcionar melhor suas promoções e serviços.


spark.sql('''SELECT DISTINCT estado_civil, COUNT(estado_civil) AS QTDE
FROM marketing
GROUP BY estado_civil''').show()

+-------------+----+
| estado_civil|QTDE|
+-------------+----+
|     solteiro| 479|
|uniao_estavel| 579|
|     viúvo(a)|  77|
|divorciado(a)| 231|
|    casado(a)| 864|
|         null|   0|
+-------------+----+



Selecionando e rankeando os clientes que mais compraram vinho acima de 1480 unidades

In [ ]:
#Procurei relacionar o estado civil com o top 3 de consumidores de vinho

spark.sql("""
  SELECT identificacao, estado_civil, qtd_vinhos
  FROM marketing 
  WHERE qtd_vinhos > 1480
  ORDER BY qtd_vinhos DESC;
""").show(3)

+-------------+-------------+----------+
|identificacao| estado_civil|qtd_vinhos|
+-------------+-------------+----------+
|          737|    casado(a)|      1493|
|         3174|uniao_estavel|      1492|
|         5536|uniao_estavel|      1492|
+-------------+-------------+----------+
only showing top 3 rows



Selecionando os clientes com doutorado e que são casados

In [ ]:
#Aqui mais uma vez o marketing poderia se beneficiar investindo em campanhas
#que atendessem melhor as expectativas desses clientes
spark.sql('''SELECT ano_nascimento, educacao, estado_civil,renda
FROM marketing
WHERE estado_civil = 'casado(a)' AND educacao = 'doutorado'
''').show()

+--------------+---------+------------+--------+
|ano_nascimento| educacao|estado_civil|   renda|
+--------------+---------+------------+--------+
|          1981|doutorado|   casado(a)| 58293.0|
|          1985|doutorado|   casado(a)| 33454.0|
|          1949|doutorado|   casado(a)| 58607.0|
|          1954|doutorado|   casado(a)| 65324.0|
|          1965|doutorado|   casado(a)| 84618.0|
|          1970|doutorado|   casado(a)| 67353.0|
|          1954|doutorado|   casado(a)| 72550.0|
|          1968|doutorado|   casado(a)| 55954.0|
|          1975|doutorado|   casado(a)| 60631.0|
|          1957|doutorado|   casado(a)|    null|
|          1970|doutorado|   casado(a)| 23626.0|
|          1983|doutorado|   casado(a)| 50150.0|
|          1945|doutorado|   casado(a)| 65846.0|
|          1980|doutorado|   casado(a)| 43974.0|
|          1961|doutorado|   casado(a)|    null|
|          1964|doutorado|   casado(a)| 41551.0|
|          1959|doutorado|   casado(a)| 33762.0|
|          1945|dout

Selecionando as colunas ano_nascimento, educacao, qtd_crianca, qtd_adolescente, guloseimas onde o ano de nascimento dos clientes é maior que 1990 e ordenando pelo consumo de guloseimas

In [ ]:
#Gostaria de saber nessa Query se existia relação das compras de doces
#e a existência de crianças ou adolescentes nas casas dos millenials

spark.sql("""
  SELECT ano_nascimento, educacao, qtd_crianca, qtd_adolescente, guloseimas
  FROM marketing 
  WHERE ano_nascimento > 1981
  ORDER BY guloseimas DESC;
""").show()

+--------------+-----------------+-----------+---------------+----------+
|ano_nascimento|         educacao|qtd_crianca|qtd_adolescente|guloseimas|
+--------------+-----------------+-----------+---------------+----------+
|          1986|        graduacao|          1|              0|       263|
|          1986|        graduacao|          0|              0|       197|
|          1984|        graduacao|          0|              0|       192|
|          1982|        graduacao|          0|              0|       185|
|          1987|        doutorado|          0|              0|       176|
|          1992|        graduacao|          0|              0|       169|
|          1986|        doutorado|          0|              0|       161|
|          1983|cursando_mestrado|          0|              0|       160|
|          1986|        graduacao|          1|              0|       149|
|          1988|        graduacao|          0|              0|       148|
|          1985|        graduacao|    

Selecionando identificacao, educacao, qtd_filhos, qtd_compras_online ordenando de forma decrescente a quantidade de compras online

In [ ]:
#Nessa query gostaria de saber se as compras online tinham alguma relação com
#a qtd de filhos e a escolaridade dos clientes
spark.sql('''
SELECT identificacao, educacao, qtd_filhos, qtd_compras_online
FROM marketing
ORDER BY qtd_compras_online DESC;
''').show()

+-------------+-----------------+----------+------------------+
|identificacao|         educacao|qtd_filhos|qtd_compras_online|
+-------------+-----------------+----------+------------------+
|         5255|        graduacao|         1|                27|
|         4619|        doutorado|         0|                27|
|        10311|        graduacao|         1|                25|
|         6237|        doutorado|         2|                23|
|         7030|        doutorado|         1|                11|
|         4910|        graduacao|         0|                11|
|          380|        graduacao|         1|                11|
|         8727|        graduacao|         1|                11|
|        10159|        graduacao|         1|                11|
|         4990|        graduacao|         0|                11|
|         4299|        graduacao|         1|                11|
|         1911|        graduacao|         0|                11|
|          796|cursando_mestrado|       

Selecionando a identificacao, educacao, qtd_filhos e qtd_compras_na_loja ordenando de forma decrescente a quantidade de compras na loja

In [ ]:
#Nessa query gostaria de saber se as compras na loja tinham alguma relação com
#a qtd de filhos e a escolaridade dos clientes
spark.sql('''
SELECT identificacao, educacao, qtd_filhos, qtd_compras_na_loja
FROM marketing
ORDER BY qtd_compras_na_loja DESC;
''').show()

+-------------+-----------------+----------+-------------------+
|identificacao|         educacao|qtd_filhos|qtd_compras_na_loja|
+-------------+-----------------+----------+-------------------+
|        10965|        graduacao|         0|                 13|
|         9305|cursando_mestrado|         0|                 13|
|          466|        graduacao|         0|                 13|
|         4114|           mestre|         0|                 13|
|         9504|           mestre|         0|                 13|
|         6312|        graduacao|         1|                 13|
|         9780|           mestre|         1|                 13|
|         1685|        doutorado|         0|                 13|
|        10299|        doutorado|         0|                 13|
|         2426|        doutorado|         0|                 13|
|        10727|        doutorado|         1|                 13|
|        10983|        graduacao|         0|                 13|
|         3856|        gr

Contando de forma distinta a quantidade de compras online de clientes de cada escolaridade destacada

In [ ]:
#Agora mais focado no E-commerce comecei a verificar especificando pelo grau de escolaridade
#Quem fazia mais compras online
spark.sql('''SELECT DISTINCT educacao, COUNT(qtd_compras_online) AS QTDE
FROM marketing
GROUP BY educacao
''').show()

+------------------+----+
|          educacao|QTDE|
+------------------+----+
| cursando_mestrado| 201|
|            mestre| 370|
|         doutorado| 485|
|         graduacao|1127|
|ensino_fundamental|  54|
+------------------+----+



Contando de forma distinta a quantidade de compras online de clientes de cada estado civil

In [ ]:
#Procurei verificar a relação do estado civil dos clientes com as compras online
# E essa query poderia ser usada em conjunto com outras pesquisa 
spark.sql('''SELECT DISTINCT estado_civil, COUNT(qtd_compras_online) AS QTDE
FROM marketing
GROUP BY estado_civil
''').show()

+-------------+----+
| estado_civil|QTDE|
+-------------+----+
|     solteiro| 479|
|uniao_estavel| 579|
|     viúvo(a)|  77|
|divorciado(a)| 231|
|    casado(a)| 864|
|         null|   7|
+-------------+----+



Selecionando o nivel de escolaridade para observar a qtd de visitas online

In [ ]:
#Intenção de verificar métricas para E-commerce com a relação da educação 
#com as visitas online. 
spark.sql('''SELECT DISTINCT educacao, COUNT (qtd_visitas_online_mes) AS QTDE
FROM marketing
GROUP BY educacao;
''').show()

+------------------+----+
|          educacao|QTDE|
+------------------+----+
| cursando_mestrado| 201|
|            mestre| 370|
|         doutorado| 485|
|         graduacao|1127|
|ensino_fundamental|  54|
+------------------+----+



Selecionando a quantidade de compras online de clientes com idade de 50 anos para cima,com base no ano de 2014.

In [ ]:
#Intenção de verificar a frequência que os clientes na maior idade
#estão fazendo compras na internet
spark.sql(''' SELECT DISTINCT (ano_nascimento), qtd_compras_online FROM marketing
WHERE ano_nascimento < 1974
ORDER BY ano_nascimento ASC;
''').show()

+--------------+------------------+
|ano_nascimento|qtd_compras_online|
+--------------+------------------+
|          1940|                 3|
|          1941|                 7|
|          1943|                 7|
|          1943|                 9|
|          1943|                 5|
|          1943|                 3|
|          1944|                 4|
|          1944|                 3|
|          1944|                 9|
|          1944|                 5|
|          1944|                 6|
|          1944|                 7|
|          1945|                10|
|          1945|                 3|
|          1945|                 8|
|          1945|                27|
|          1945|                 6|
|          1946|                 4|
|          1946|                 0|
|          1946|                 7|
+--------------+------------------+
only showing top 20 rows



Relacionando algumas caracteristicas dos clientes com a última vez que fez compras, quantidade de compras online e as visitas online no mês

In [ ]:
#Intenção de verificar se dependendo do estado civil ou educação, os dias que os clientes ficam sem comprar
#Teria relação com a quantidade de compras no mês
spark.sql('''
SELECT identificacao, educacao, estado_civil, dias_desde_ult_compras, qtd_compras_online, qtd_visitas_online_mes
FROM marketing
''').show()

+-------------+------------------+-------------+----------------------+------------------+----------------------+
|identificacao|          educacao| estado_civil|dias_desde_ult_compras|qtd_compras_online|qtd_visitas_online_mes|
+-------------+------------------+-------------+----------------------+------------------+----------------------+
|         5524|         graduacao|     solteiro|                    58|                 8|                     7|
|         2174|         graduacao|     solteiro|                    38|                 1|                     5|
|         4141|         graduacao|uniao_estavel|                    26|                 8|                     4|
|         6182|         graduacao|uniao_estavel|                    26|                 2|                     6|
|         5324|         doutorado|    casado(a)|                    94|                 5|                     5|
|         7446|            mestre|uniao_estavel|                    16|                 